In [70]:
import pandas as pd
import numpy as np
from datetime import datetime
from pandas.api.types import is_numeric_dtype


In [232]:
players = pd.read_csv("data/players.csv")

In [13]:
players.dtypes

nflId                        int64
height                      object
weight                       int64
birthDate           datetime64[ns]
collegeName                 object
officialPosition            object
displayName                 object
age                        float64
dtype: object

In [233]:
today = datetime.today()
players['birthDate'] = pd.to_datetime(players['birthDate'])

players['age'] = players['birthDate'].apply(
               lambda x: today.year - x.year - 
               ((today.month, today.day) < (x.month, x.day)) 
               )

In [5]:
players.head()

,nflId,height,weight,birthDate,collegeName,officialPosition,displayName,age
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady,45.0
1,28963,6-5,240,1982-03-02,"Miami, O.",QB,Ben Roethlisberger,40.0
2,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters,41.0
3,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers,39.0
4,30078,6-2,228,1982-11-24,Harvard,QB,Ryan Fitzpatrick,40.0


In [6]:
players[["officialPosition", "weight"]].groupby("officialPosition").mean().sort_values("weight")

,weight
officialPosition,
CB,193.109375
DB,198.000000
WR,199.366667
FS,202.361111
SS,204.231884
RB,214.190476
QB,220.733333
MLB,233.259259
ILB,235.567901


In [171]:
players[["officialPosition", "age"]].groupby("officialPosition").describe()

age                                                    
                  count       mean       std   min   25%   50%    75%   max
officialPosition                                                           
C                  49.0  28.673469  2.988783  24.0  26.0  29.0  31.00  37.0
CB                158.0  27.556962  2.691089  23.0  26.0  27.0  29.00  35.0
DB                  1.0  24.000000       NaN  24.0  24.0  24.0  24.00  24.0
DE                122.0  28.008197  3.008242  23.0  26.0  27.0  30.00  36.0
DT                 98.0  27.979592  2.743238  24.0  26.0  28.0  29.75  35.0
FB                 15.0  27.800000  2.007130  25.0  26.0  28.0  29.50  31.0
FS                 62.0  27.741935  3.056493  24.0  25.0  27.0  29.00  35.0
G                  89.0  27.898876  2.717974  23.0  25.0  28.0  30.00  34.0
ILB                66.0  27.954545  2.433306  24.0  26.0  28.0  30.00  34.0
LB                  1.0  33.000000       NaN  33.0  33.0  33.0  33.00  33.0
MLB                25.0  26.600000  2.327373  23.0  25.0  26.0  27.00  33.0
NT                 29.0  28.827586  3.024530  24.0  27.0  28.0  30.00  37.0
OLB               115.0  27.800000  2.629789  23.0  26.0  27.0  30.00  34.0
QB                 53.0  30.245283  4.937711  24.0  26.0  29.0  34.00  45.0
RB                107.0  27.065421  2.348307  23.0  25.0  27.0  28.00  34.0
SS                 63.0  27.650794  2.695030  23.0  26.0  27.0  29.50  35.0
T                 102.0  28.715686  3.549864  23.0  26.0  28.0  31.00  41.0
TE                106.0  28.254717  2.885304  23.0  26.0  28.0  30.00  38.0
WR                186.0  27.548387  2.788068  23.0  25.0  27.0  29.00  37.0

In [166]:
# write function to group by one variable and provide numeric summary statistics of another
# If the group by variable is numeric, provide summaries by quantile
# Else, provide summaries for the top 5 levels with all data in an "other" bucket


def get_summary(data, group_var, output_var):
    if not isinstance(data[output_var][len(data[output_var])-1], (np.int64, int, float, complex)):
        raise ValueError("output_var must be type int, float or complex")
    else:
        # numeric - deciles
        if isinstance(data[group_var][len(data[group_var])-1], (np.int64, int, float, complex)):
            var_nm = [group_var, '_quantile']
            string = ""
            
            step_df = data.loc[:, (group_var, output_var)]
            step_df[string.join(var_nm)] = pd.qcut(data[group_var], 4)
            out_df = step_df[[string.join(var_nm), output_var]].groupby(string.join(var_nm)).describe()
       
        # categorical - top 5 levels
        else:
            var_nm = [group_var, '_grp']
            string = ""
            
            top_levels = data[[group_var]].groupby(group_var).value_counts().nlargest(5)
            step_df = data.loc[:, (group_var, output_var)]
            step_df[string.join(var_nm)] = step_df[group_var].apply(lambda x: (x if x in top_levels else "other"))
            out_df = step_df[[string.join(var_nm), output_var]].groupby(string.join(var_nm)).describe()

        return(out_df)
    


In [169]:
get_summary(players, "weight", "age")

age                                                   
                  count       mean       std   min   25%   50%   75%   max
weight_quantile                                                           
(157.999, 205.0]  368.0  27.635870  2.796069  23.0  26.0  27.0  29.0  37.0
(205.0, 236.0]    353.0  27.631728  3.050766  23.0  25.0  27.0  29.0  45.0
(236.0, 292.0]    364.0  27.986264  2.870449  23.0  26.0  27.0  30.0  40.0
(292.0, 380.0]    362.0  28.497238  3.082655  23.0  26.0  28.0  30.0  41.0

In [170]:
get_summary(players, "officialPosition", "age")

age                                                   
                      count       mean       std   min   25%   50%   75%   max
officialPosition_grp                                                          
CB                    158.0  27.556962  2.691089  23.0  26.0  27.0  29.0  35.0
DE                    122.0  28.008197  3.008242  23.0  26.0  27.0  30.0  36.0
OLB                   115.0  27.800000  2.629789  23.0  26.0  27.0  30.0  34.0
RB                    107.0  27.065421  2.348307  23.0  25.0  27.0  28.0  34.0
WR                    186.0  27.548387  2.788068  23.0  25.0  27.0  29.0  37.0
other                 759.0  28.246377  3.144902  23.0  26.0  28.0  30.0  45.0

In [239]:
# Feature Engineering - Binning
# Age buckets
# Age to Offset
# Position Groups

## Read Mapping Table
players_map = pd.read_csv("data/players_map.csv", usecols = ['variable', 'curr_val', 'new_val'])

# Numerical Binning
### inputs:
#          dataframe for mapping
#          mapping table with names "variable", "curr_val" and "new_val"
#          variable name in dataframe
#          variable name in mapping table

### Notes:
#     lower bound inclusive i.e. curr_val step from 30 to 35, row 30 will include 30 to 34 for whole number ranges


def num_bin(df, map_df, var, map_var):
    var_nm = [map_var, '_bin_n']
    string = ""
    
    label_names = map_df[map_df["variable"] == map_var][["new_val"]]
    label_list = label_names["new_val"].tolist()

    cut_points = map_df[map_df["variable"] == map_var][["curr_val"]]
    cut_list = cut_points["curr_val"].tolist()

    df[string.join(var_nm)] = pd.cut(df[var], cut_list, labels = label_list[:-1], ordered = False)
        
    return df


# Categorical Grouping

## Notes:
#     will throw an error if a value is not in the mapping table under curr_val

def cat_bin(df, map_df, var, map_var):
    var_nm = [map_var, '_bin_c']
    string = ""
    
    map_dict = dict(zip(map_df[map_df["variable"] == map_var].curr_val, map_df[map_df["variable"] == map_var].new_val))
    df[var] = df[var].astype('category')
        
    df[string.join(var_nm)] = df[var].apply(lambda x : map_dict[x])
    
    return df

    

In [240]:
cat_bin(players, players_map, "officialPosition", "officialPosition")

,nflId,height,weight,birthDate,collegeName,officialPosition,displayName,age,officialPosition_bin_c
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady,45.0,OTHER
1,28963,6-5,240,1982-03-02,"Miami, O.",QB,Ben Roethlisberger,40.0,OTHER
2,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters,41.0,OTHER
3,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers,39.0,OTHER
4,30078,6-2,228,1982-11-24,Harvard,QB,Ryan Fitzpatrick,40.0,OTHER
...,...,...,...,...,...,...,...,...,...
1674,53991,6-1,320,NaT,NaN,DT,Forrest Merrill,NaN,OTHER
1675,53994,6-5,300,NaT,NaN,C,Ryan McCollum,NaN,OTHER
1676,53999,6-4,312,NaT,NaN,DT,Jack Heflin,NaN,OTHER
1677,54006,6-6,330,NaT,NaN,T,Jake Curhan,NaN,OTHER


In [236]:
players[players['officialPosition']  == 'WR']

,nflId,height,weight,birthDate,collegeName,officialPosition,displayName,age,officialPosition_bin_c
9,33130,5-10,175,1986-12-01,California,WR,DeSean Jackson,36.0,WR
13,33566,5-11,185,1985-11-02,Texas Tech,WR,Danny Amendola,37.0,WR
37,35521,5-11,180,1987-03-17,Southern Methodist,WR,Emmanuel Sanders,35.0,WR
39,35527,5-11,195,1988-01-09,Citadel,WR,Andre Roberts,35.0,WR
43,35634,5-10,185,1988-07-10,Central Michigan,WR,Antonio Brown,34.0,WR
...,...,...,...,...,...,...,...,...,...
1626,53633,5-10,190,NaT,NaN,WR,Shi Smith,NaN,WR
1627,53634,6-3,224,NaT,NaN,WR,Racey McMath,NaN,WR
1638,53658,6-5,205,NaT,NaN,WR,Mike Strachan,NaN,WR
1645,53678,6-3,220,NaT,NaN,WR,Ben Skowronek,NaN,WR


In [225]:
dict(zip(players_map[players_map["variable"] == "officialPosition"].curr_val, players_map[players_map["variable"] == "officialPosition"].new_val))

{'C': 'OTHER',
 'CB': 'CB',
 'DB': 'OTHER',
 'DE': 'DE',
 'DT': 'OTHER',
 'FB': 'OTHER',
 'FS': 'OTHER',
 'G': 'OTHER',
 'ILB': 'OTHER',
 'LB': 'OTHER',
 'MLB': 'OTHER',
 'NT': 'OTHER',
 'OLB': 'OLB',
 'QB': 'OTHER',
 'RB': 'RB',
 'SS': 'OTHER',
 'T': 'OTHER',
 'TE': 'OTHER',
 'WR': 'WR'}